This notebook contains the work-in-progress (WIP) scrap for the `imbed` project.

It is not meant to be run by all.


# The k-d to 2-d problem

After embedding your objects of interest into k-dimensional vectors, you might further embed them into two or three dimensions for visualization. This process inevitably loses some information and introduces distortions, but it's still valuable. The initial conversion of texts or images into numerical vectors also involved information loss, yet it was useful for your specific goals.

Dimensionality reduction typically results in losing some details. The key is to preserve the essential information (the "signal") and minimize the loss of less important details (the "noise"). Done well, this can be beneficial, especially if it enhances the signal-to-noise ratio.

This principle also applies to compressing vectors for visualization. The aim is to present complex data usefully and intuitively, enabling analysts to spot patterns and gain insights. It's a balance between maintaining utility and practical implementation when transforming k-dimensional data into two-dimensional forms.



# Scrap